In [6]:
#Importing the base modules
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms  #Has lots of pre-trained models 
#Modules for training
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
#Other hellpful libraries
import numpy as np
import matplotlib.pyplot as plt

import time
import os
import shutil
# import copy

#plt.ion()  



In [12]:
source_folder='./hemanth/tdanet/results/tda_bird'
destination_folder1='./data/train/authentic'
destination_folder2='./data/train/generated'

files = os.listdir(source_folder)

for file in files:
    print(file.split(".")[0].split("_")[-1])
    x=file.split(".")[0].split("_")[-1]
    if x=='0':
        shutil.copy(os.path.join(source_folder,file), os.path.join(destination_folder2,file))
    elif x=='truth':
        shutil.copy(os.path.join(source_folder,file), os.path.join(destination_folder1,file))
    

0
mask
mask
truth
mask
truth
mask
mask
mask
mask
truth
truth
0
0
0
truth
truth
truth
mask
truth
mask
0
truth
truth
0
0
0
truth
0
mask
truth
mask
truth
0
truth
0
0
truth
0
truth
truth
mask
mask
mask
0
0
mask
0
mask
0
mask
mask
mask
truth
truth
0
0
truth
truth
truth
0
truth
mask
mask
0
0
truth
truth
truth
0
mask
mask
mask
truth
mask
truth
mask
mask
mask
truth
0
0
0
truth
mask
mask
truth
mask
truth
mask
truth
truth
mask
0
truth
truth
truth
0
mask
0
truth
mask
mask
truth
mask
mask
mask
0
0
mask
0
mask
truth
0
0
0
0
0
truth
mask
mask
0
truth
truth
mask
truth
mask
0
mask
0
truth
truth
mask
truth
0
truth
mask
0
0
0
mask
0
truth
truth
truth
mask
truth
0
mask
0
mask
mask
0
mask
0
mask
0
truth
0
0
0
0
mask
mask
mask
truth
0
0
truth
0
mask
truth
mask
mask
mask
mask
0
mask
0
truth
truth
mask
mask
truth
0
truth
mask
truth
mask
mask
mask
0
mask
truth
0
0
0
0
mask
0
truth
truth
0
truth
truth
truth
mask
0
mask
mask
mask
mask
0
truth
0
mask
mask
0
0
truth
mask
mask
mask
mask
mask
0
0
truth
truth
0
trut

In [14]:
# sf1='./data/train/authentic'
# sf2='./data/train/generated/'
# df1='./data/validation/authentic'
# df2='./data/validation/generated'

# files1 = os.listdir(sf1)
# files2 = os.listdir(sf2)
# c1=0
# for file in files1:
#     if c1==600:
#         break
#     shutil.move(os.path.join(sf1,file), os.path.join(df1,file))
#     c1+=1
# c2=0
# for file in files2:
#     if c2==600:
#         break
#     shutil.move(os.path.join(sf2,file), os.path.join(df2,file))
#     c2+=1

# files1 = os.listdir(sf1)
# files2 = os.listdir(sf2)

# df3='./data/test/authentic'
# df4='./data/test/generated'
# c3=0
# for file in files1:
#     if c3==600:
#         break
#     shutil.move(os.path.join(sf1,file), os.path.join(df3,file))
#     c3+=1
# c4=0
# for file in files2:
#     if c4==600:
#         break
#     shutil.move(os.path.join(sf2,file), os.path.join(df4,file))
#     c4+=1

In [15]:
data_dir = './data'
train_dir = 'train'
val_dir = 'validation'
test_dir = 'test'

data_transforms = {  
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
        transforms.RandomRotation(degrees=(0, 180)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([      
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    'train': datasets.ImageFolder(    
        os.path.join(data_dir, train_dir), 
        transform=data_transforms['train']
    ),
    'val': datasets.ImageFolder(    
        os.path.join(data_dir, val_dir), 
        transform=data_transforms['val']
    ),
    'test': datasets.ImageFolder(    
        os.path.join(data_dir, test_dir), 
        transform=data_transforms['test']
    ),
}

dataloaders = {    
    'train': torch.utils.data.DataLoader(
        image_datasets['train'], batch_size=2,
        shuffle=True, num_workers=7
    ),
    'val': torch.utils.data.DataLoader(
        image_datasets['val'], batch_size=2,
        shuffle=True, num_workers=7
    ),
    'test': torch.utils.data.DataLoader(
        image_datasets['test'], batch_size=2,
        shuffle=True, num_workers=7
    ),
}

for x in image_datasets.keys():
    print(x,len(image_datasets[x]))
    
print("Classes: ", image_datasets["train"].classes)

train 9588
val 1200
test 1200
Classes:  ['authentic', 'generated']


## Model Training

I used pretrained models available with pytorch. Since they were trained on imageNet dataset with 1000 classes $\therefore$ they have 1000 output features in the end, so I willl replace the last fully connected layer of the model with a new one with 2 output features as per our dataset.

Now with use of pretrained models, I choose to train all the layers instead of just the last fully connected layers as I have previously got better results with it.

For optimizer, I used SGD since it is better for image classification tasks than compared to Adam and has faster convergence(reference: https://opt-ml.org/papers/2021/paper53.pdf)

For Learning rate I set an initial learning rate of 10^-4 and used a scheduler which multiplies the learning rate with a certain $\gamma$ after every set epochs, in this case, multiplies by 0.1 after every 5 epochs

In [16]:
def train(name, model, criterion, optimizer, scheduler, num_epochs=10):
    start = time.time()
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    history = {'ta':[],'tl':[],'va':[],'vl':[]}
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        print('----------')
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0

        #Start the training
        model.train(True)        
        for i, data in enumerate(dataloaders['train']):
            if(i%100 == 0):
                print("\rTraining batch {}/{}".format(i, len(dataloaders['train'])/2), end='', flush=True)
                print()
            if(i >= len(dataloaders['train'])/2):
                break
            inputs, labels = data
            (inputs, labels) = ((Variable(inputs), Variable(labels)),(Variable(inputs.cuda()), Variable(labels.cuda())))[torch.cuda.is_available()]
            optimizer.zero_grad()
            outputs = model(inputs)
            preds = torch.max(outputs.data, 1)[1]
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        avg_loss = loss_train * 2 / 20000
        avg_acc = acc_train * 2 / 20000
        model.train(False)

        #Start the evaluation
        model.eval()
        for i, data in enumerate(dataloaders['val']):
            if i % 100 == 0:
                print(f"\rValidation batch {i}/{len(dataloaders['val'])}", end='', flush=True)
            with torch.no_grad():     #This is to not store any gradients as we won't be doing backpropagation now.
                inputs, labels = data
                (inputs, labels) = ((Variable(inputs), Variable(labels)),(Variable(inputs.cuda()), Variable(labels.cuda())))[torch.cuda.is_available()]
                optimizer.zero_grad()                
                outputs = model(inputs)                
                preds = torch.max(outputs.data, 1)[1]
                loss = criterion(outputs, labels)                
                loss_val += loss.item()
                acc_val += torch.sum(preds == labels.data)                
                del inputs, labels, outputs, preds
                torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / 4000
        avg_acc_val = acc_val / 4000

        #Save the model
        torch.save(model.state_dict(), f'{name}_{epoch}.pt')
        #Print the results and store them
        history['ta'].append(avg_acc)
        history['tl'].append(avg_loss)
        history['va'].append(avg_acc_val)
        history['vl'].append(avg_loss_val)
        print(f"\nEpoch {epoch} result: ")
        print(f"Training loss: [{avg_loss}]")
        print(f"Training acc: [{avg_acc}]")
        print(f"Validation loss: [{avg_loss_val}]")
        print(f"Validation acc: [{avg_acc_val}]")
        print('----------\n')
        
    train_time = time.time() - start
    print(f"\nTraining time: {train_time//60}m {train_time%60}s")
    return history

In [20]:
import torch.nn as nn
import torchvision.models as models

# Initialize ResNeXt-101 32x8d pre-trained model
resnext = models.resnext101_32x8d(weights='IMAGENET1K_V1')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnext.to(device)

# Modify Fully Connected (fc) Layer for Binary Classification
resnext.fc = nn.Sequential(
    resnext.fc,
    nn.Linear(in_features=1000, out_features=512, bias=True),
    nn.ReLU(),  # Add a ReLU activation function
    nn.Linear(in_features=512, out_features=256, bias=True),
    nn.ReLU(),  # Add another ReLU activation function
    nn.Linear(in_features=256, out_features=2, bias=True),  # Output size set to 2 for binary classification
)

# Print the Modified Model
print(resnext)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(resnext.parameters(), lr=0.0001, momentum=0.8) #initital learning rate of 10^-4
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)  #Scheduler
train('resnet34',resnext, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

In [14]:
# SWIN TRANSFORMER
import torch.nn as nn
import torchvision.models as models

swint = models.swin_v2_b(weights='IMAGENET1K_V1')

# Modify the head for binary classification (2 classes)
swint.head = nn.Sequential(
    swint.head,
    nn.Linear(in_features=1000, out_features=512, bias=True),
    nn.Linear(in_features=512, out_features=256, bias=True),
    nn.Linear(in_features=256, out_features=2, bias=True),
)
print(swint)

SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (proj): Linear(in_features=128, out_features=128, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=4, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(appro

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(swint.parameters(), lr=0.0001, momentum=0.8)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
train('SwinT', swint, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)